In [21]:
# Data cleaning:
import re
import codecs
import pandas as pd

def decode(input_filepath, output_filepath):
    with codecs.open(input_filepath, "r") as input_file:
        with codecs.open(output_filepath, "w", encoding="utf-8") as output_file:
            for line in input_file:
                file_content = re.sub("\t", ",", line)
                output_file.write(file_content)

decode("pokemon2-8.tsv", "pokemon.csv")

def df_import(csv_file):
    poke_df = pd.read_csv(csv_file, index_col="national_number")
    return poke_df

poke_df = df_import('pokemon.csv')

def df_drop_add(dataframe):
    """
    ## df_drop_add(dataframe)
    removes a specific list of columns & adds a 'wins' and 'losses' column

    *dataframe:
    - takes a pandas dataframe
    """
    # Columns to drop from dataframe
    dataframe.drop(columns=[
        'japanese_name',
        'percent_male',
        'percent_female',
        'capture_rate',
        'base_egg_steps',
        'evochain_0',
        'evochain_1',
        'evochain_2',
        'evochain_3',
        'evochain_4',
        'evochain_5',
        'evochain_6',
        'gigantamax',
        'mega_evolution',
        'mega_evolution_alt',
    ],   
        axis=1,
        inplace=True,
    )
    # Columns to add to dataframe
    col_list=['wins','losses']
    for col in col_list:
        if col not in dataframe.columns:
            dataframe['wins']=0
            dataframe['losses']=0
        else:
            pass
    # rename column "english_name" to "name"
    dataframe.rename(columns={"english_name": "name"}, inplace=True)
    return dataframe
df_drop_add(poke_df)   
poke_df
# poke_df.dtypes

,gen,name,primary_type,secondary_type,classification,height_m,weight_kg,hp,attack,defense,...,against_dragon,against_dark,against_steel,against_fairy,is_sublegendary,is_legendary,is_mythical,description,wins,losses
national_number,,,,,,,,,,,,,,,,,,,,,
1,I,Bulbasaur,grass,poison,Seed Pokémon,0.7,6.9,45,49,49,...,1.0,1.0,1.0,0.5,0,0,0,There is a plant seed on its back right from t...,0,0
2,I,Ivysaur,grass,poison,Seed Pokémon,1.0,13.0,60,62,63,...,1.0,1.0,1.0,0.5,0,0,0,"When the bulb on its back grows large, it appe...",0,0
3,I,Venusaur,grass,poison,Seed Pokémon,2.0,100.0,80,82,83,...,1.0,1.0,1.0,0.5,0,0,0,Its plant blooms when it is absorbing solar en...,0,0
4,I,Charmander,fire,NaN,Lizard Pokémon,0.6,8.5,39,52,43,...,1.0,1.0,0.5,0.5,0,0,0,It has a preference for hot things. When it ra...,0,0
5,I,Charmeleon,fire,NaN,Flame Pokémon,1.1,19.0,58,64,58,...,1.0,1.0,0.5,0.5,0,0,0,"It has a barbaric nature. In battle, it whips ...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,VIII,Regieleki,electric,NaN,Electron Pokémon,1.2,145.0,80,100,50,...,1.0,1.0,0.5,1.0,1,0,0,This Pokémon is a cluster of electrical energy...,0,0
895,VIII,Regidrago,dragon,NaN,Dragon Orb Pokémon,2.1,200.0,200,100,50,...,2.0,1.0,1.0,2.0,1,0,0,An academic theory proposes that Regidrago’s a...,0,0
896,VIII,Glastrier,ice,NaN,Wild Horse Pokémon,2.2,800.0,100,145,130,...,1.0,1.0,2.0,1.0,1,0,0,Glastrier emits intense cold from its hooves. ...,0,0


In [22]:
# load to big query

from google.cloud import bigquery
import logging
from logging import INFO
import pandas as pd
import pandas_gbq
import sys

logging.basicConfig(format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
                    level=INFO,
                    stream=sys.stderr)
logger: logging.Logger = logging

# Instantiate big query client api which will create a dataset
client = bigquery.Client()
# Tell the client to use "poke_battler_table" as the dataset name to create in the project
dataset_id = "{}.poke_battler_table".format(client.project)
# Pass dataset_id to bigquery's Dataset class to build a reference
dataset = bigquery.Dataset(dataset_id)
# Assign the datasets server location to US
dataset.location = "US"
# Tell the client to create the dataset on google big query with the completed information
dataset = client.create_dataset(dataset, exists_ok=True, timeout=30)
# If successful, log the creation of the dataset
logger.info("Created dataset {}.{}.".format(client.project, dataset.dataset_id))



# Project to look for when creating a table,
project_id = "deb-01-346205"
# dataset to insert the table into
table_id = "poke_battler_table.pokemon"

# Loading transformed dataframe into google big query with the specified project/dataset as targets and a specified table schema.
logger.info("Loading dataframe to {}...".format(dataset.dataset_id))
pandas_gbq.to_gbq(poke_df, table_id, project_id=project_id, if_exists="replace", api_method="load_csv", table_schema=[
    {'name': 'national_number', 'type': 'INT64'}, 
    {'name': 'gen', 'type': 'STRING'}, 
    {'name': 'name', 'type': 'STRING'}, 
    {'name': 'primary_type', 'type': 'STRING'}, 
    {'name': 'secondary_type', 'type': 'STRING'}, 
    {'name': 'classification', 'type': 'STRING'}, 
    {'name': 'height_m', 'type': 'FLOAT64'}, 
    {'name': 'weight_kg', 'type': 'FLOAT64'}, 
    {'name': 'hp', 'type': 'INT64'}, 
    {'name': 'attack', 'type': 'INT64'}, 
    {'name': 'defense', 'type': 'INT64'}, 
    {'name': 'sp_attack', 'type': 'INT64'}, 
    {'name': 'sp_defense', 'type': 'INT64'}, 
    {'name': 'speed', 'type': 'INT64'}, 
    {'name': 'abilities_0', 'type': 'STRING'}, 
    {'name': 'abilities_1', 'type': 'STRING'}, 
    {'name': 'abilities_2', 'type': 'STRING'},
    {'name': 'abilities_hidden', 'type': 'STRING'},
    {'name': 'against_normal', 'type': 'FLOAT64'},
    {'name': 'against_fire', 'type': 'FLOAT64'},
    {'name': 'against_water', 'type': 'FLOAT64'},
    {'name': 'against_electric', 'type': 'FLOAT64'},
    {'name': 'against_grass', 'type': 'FLOAT64'},
    {'name': 'against_ice', 'type': 'FLOAT64'},
    {'name': 'against_fighting', 'type': 'FLOAT64'},
    {'name': 'against_poison', 'type': 'FLOAT64'},
    {'name': 'against_ground', 'type': 'FLOAT64'},
    {'name': 'against_flying', 'type': 'FLOAT64'},
    {'name': 'against_psychic', 'type': 'FLOAT64'},
    {'name': 'against_bug', 'type': 'FLOAT64'},
    {'name': 'against_rock', 'type': 'FLOAT64'},
    {'name': 'against_ghost', 'type': 'FLOAT64'},
    {'name': 'against_dragon', 'type': 'FLOAT64'},
    {'name': 'against_dark', 'type': 'FLOAT64'},
    {'name': 'against_steel', 'type': 'FLOAT64'},
    {'name': 'against_fairy', 'type': 'FLOAT64'},
    {'name': 'is_sublegendary', 'type': 'INT64'},
    {'name': 'is_legendary', 'type': 'INT64'},
    {'name': 'is_mythical', 'type': 'INT64'},
    {'name': 'description', 'type': 'STRING'},
    {'name': 'wins', 'type': 'INT64'},
    {'name': 'losses', 'type': 'INT64'}])
logger.info(f"Successfully loaded poke_df into " + "{}.".format(dataset.dataset_id))

[INFO ][2022-06-07 11:14:33,517][1112897290:0026] : Created dataset deb-01-346205.poke_battler_table.
[INFO ][2022-06-07 11:14:33,518][1112897290:0036] : Loading dataframe to poke_battler_table...
898 out of 898 rows loaded.,673][gbq:0606] : 
[INFO ][2022-06-07 11:14:38,060][1112897290:0081] : Successfully loaded poke_df into poke_battler_table.
